有时，我们并不需要在路径操作函数中使用依赖项的返回值。

或者说，有些依赖项不返回值。

但仍要执行或解析该依赖项。

对于这种情况，不必在声明路径操作函数的参数时使用 Depends，而是可以在路径操作装饰器中添加一个由 dependencies 组成的 list。

路径操作装饰器支持可选参数 ~ dependencies。

该参数的值是由 Depends() 组成的 list：

In [1]:
import uvicorn
from fastapi import Depends, FastAPI, Header, HTTPException
app = FastAPI()


async def verify_token(x_token: str = Header()):
    if x_token != "fake-super-secret-token":
        raise HTTPException(status_code=400, detail="X-Token header invalid")


async def verify_key(x_key: str = Header()):
    if x_key != "fake-super-secret-key":
        raise HTTPException(status_code=400, detail="X-Key header invalid")
    return x_key


@app.get("/items/", dependencies=[Depends(verify_token), Depends(verify_key)])
async def read_items():
    return [{"item": "Foo"}, {"item": "Bar"}]

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [13712]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:65369 - "GET /items/ HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:65370 - "GET /items/ HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:65379 - "GET /items/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:65393 - "GET /items/ HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:65397 - "GET /items/ HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:65408 - "GET /items/ HTTP/1.1" 400 Bad Request


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [13712]


In [ ]:
url = 'http://127.0.0.1:8009/items/' 
headers = {"x-token": "fake-super-secret-token","x-key": "fake-super-secret-key"} 
res = requests.get(url, headers=headers) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '[{"item":"Foo"},{"item":"Bar"}]'

In [ ]:
url = 'http://127.0.0.1:8009/items/' 
headers = {"x-token": "fake-super-secret-token1","x-key": "fake-super-secret-key"} 
res = requests.get(url, headers=headers) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"detail":"X-Token header invalid"}'

In [ ]:
url = 'http://127.0.0.1:8009/items/' 
headers = {"x-token": "fake-super-secret-token","x-key": "fake-super-secret-key1"} 
res = requests.get(url, headers=headers) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"detail":"X-Key header invalid"}'

路径操作装饰器依赖项（以下简称为“路径装饰器依赖项”）的执行或解析方式和普通依赖项一样，但就算这些依赖项会返回值，它们的值也不会传递给路径操作函数。

有些编辑器会检查代码中没使用过的函数参数，并显示错误提示。

在路径操作装饰器中使用 dependencies 参数，可以确保在执行依赖项的同时，避免编辑器显示错误提示。

使用路径装饰器依赖项还可以避免开发新人误会代码中包含无用的未使用参数。

触发异常

路径装饰器依赖项与正常的依赖项一样，可以 raise 异常：

返回值

无论路径装饰器依赖项是否返回值，路径操作都不会使用这些值。

因此，可以复用在其他位置使用过的、（能返回值的）普通依赖项，即使没有使用这个值，也会执行该。赖项：